In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
from PIL import Image, ImageTk

# Helper Functions

def apply_sobel(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    sobel = cv2.convertScaleAbs(sobelx + sobely)
    return cv2.cvtColor(sobel, cv2.COLOR_GRAY2BGR)

def apply_canny(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def apply_gaussian_blur(img):
    return cv2.GaussianBlur(img, (15, 15), 0)

def apply_median_blur(img):
    return cv2.medianBlur(img, 15)

def apply_histogram_equalization(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    return cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)

def apply_sharpening(img):
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    return cv2.filter2D(img, -1, kernel)

def apply_face_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    if face_cascade.empty():
        messagebox.showerror("Error", "Failed to load Haar Cascade XML file.")
        return img

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    if len(faces) == 0:
        messagebox.showinfo("Face Detection", "No faces detected in the image.")
    else:
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return img

def apply_pencil_sketch(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    inv_img = 255 - gray_img
    blur_img = cv2.GaussianBlur(inv_img, (21, 21), 0)
    sketch = cv2.divide(gray_img, 255 - blur_img, scale=256.0)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)

def apply_cartoon(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(img, 9, 300, 300)
    cartoon = cv2.bitwise_and(color, color, mask=edges)
    return cartoon

FILTERS = {
    "Sobel Edge Detection": apply_sobel,
    "Canny Edge Detection": apply_canny,
    "Gaussian Blur": apply_gaussian_blur,
    "Median Blur": apply_median_blur,
    "Histogram Equalization": apply_histogram_equalization,
    "Sharpening": apply_sharpening,
    "Face Detection": apply_face_detection,
    "Pencil Sketch": apply_pencil_sketch,
    "Cartoon Effect": apply_cartoon
}

class ImageFilterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Filter Utility")
        self.root.geometry("1000x600")
        self.root.configure(bg='#f0f0f0')

        self.image = None
        self.processed_image = None

        # Frames
        self.panel_original = tk.Label(root, bg='#e0e0e0', relief="groove")
        self.panel_original.place(x=50, y=50, width=400, height=400)

        self.panel_processed = tk.Label(root, bg='#e0e0e0', relief="groove")
        self.panel_processed.place(x=550, y=50, width=400, height=400)

        control_frame = tk.Frame(root, bg='#d9d9d9', bd=2, relief="raised")
        control_frame.place(x=50, y=470, width=900, height=100)

        # Filter Dropdown
        self.filter_var = tk.StringVar(root)
        self.filter_var.set("Sobel Edge Detection")

        filter_label = tk.Label(control_frame, text="Select Filter:", bg='#d9d9d9', font=('Arial', 10, 'bold'))
        filter_label.grid(row=0, column=0, padx=10, pady=10)

        filter_menu = ttk.Combobox(control_frame, textvariable=self.filter_var, values=list(FILTERS.keys()), width=30)
        filter_menu.grid(row=0, column=1, padx=10)

        # Buttons
        style = ttk.Style()
        style.configure('TButton', font=('Arial', 10, 'bold'), padding=5)

        btn_load = ttk.Button(control_frame, text="Load Image", command=self.load_image)
        btn_load.grid(row=0, column=2, padx=10)

        btn_apply = ttk.Button(control_frame, text="Apply Filter", command=self.apply_filter)
        btn_apply.grid(row=0, column=3, padx=10)

        btn_save = ttk.Button(control_frame, text="Save Image", command=self.save_image)
        btn_save.grid(row=0, column=4, padx=10)

    def load_image(self):
        path = filedialog.askopenfilename()
        if path:
            self.image = cv2.imread(path)
            self.show_image(self.image, self.panel_original)

    def apply_filter(self):
        if self.image is None:
            messagebox.showerror("Error", "Load an image first")
            return
        filter_func = FILTERS[self.filter_var.get()]
        self.processed_image = filter_func(self.image.copy())
        self.show_image(self.processed_image, self.panel_processed)

    def save_image(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No processed image to save")
            return
        path = filedialog.asksaveasfilename(defaultextension=".jpg")
        if path:
            cv2.imwrite(path, self.processed_image)
            messagebox.showinfo("Saved", f"Image saved to {path}")

    def show_image(self, img, panel):
        b, g, r = cv2.split(img)
        img = cv2.merge((r, g, b))
        im = Image.fromarray(img)
        im = im.resize((400, 400))
        imgtk = ImageTk.PhotoImage(image=im)
        panel.imgtk = imgtk
        panel.config(image=imgtk)

if __name__ == '__main__':
    root = tk.Tk()
    app = ImageFilterApp(root)
    root.mainloop()
